In [ ]:
import re
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from datetime import datetime
import requests

In [ ]:
# Path to auth log (works on Unix/Linux)
LOG_PATH = "/var/log/auth.log"

# Store parsed results
failed_logins = []
